# 라이브러리

In [16]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow
import os, re 
import numpy as np
import tensorflow as tf
print(tensorflow.__version__)
from sklearn.model_selection import train_test_split

2.6.0


## Step 2. 데이터 읽어오기
glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]']


In [4]:
print(raw_corpus[:9])

['', '', '[Spoken Intro:]', 'You ever want something ', "that you know you shouldn't have ", "The more you know you shouldn't have it, ", 'The more you want it ', 'And then one day you get it, ', "It's so good too "]


## Step 3. 데이터 정제
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [5]:
# enumerate() 함수를 이용하여 raw_corpus list 내에 저장된 문장과 그 문장의 인덱스를 반환 (인덱스, 문장 순)
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)


[Spoken Intro:]
You ever want something 
that you know you shouldn't have 
The more you know you shouldn't have it, 
The more you want it 
And then one day you get it, 
It's so good too 
But it's just like my girl 


In [6]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [7]:
# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


@_를넣어도 저절로 빼주는 문장완성.

In [18]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence.split()) >= 13: continue  #15개 이하(start,end포함)
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> spoken intro <end>',
 '<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>']

## Step 4. 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!



In [11]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12,000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12,000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    # maxlen 선택적 Int, 모든 시퀀스의 최대 길이. 설정하지 않는 경우,시퀀스는 가장 긴 개별 길이로 채워집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2701 2584 ...    0    0    0]
 [   2    7  156 ...    0    0    0]
 [   2   17    7 ...    0    0    0]
 ...
 [   2  311    1 ...    0    0    0]
 [   5   34   45 ... 1161  143    3]
 [   5   34   45 ... 1161  143    3]] <keras_preprocessing.text.Tokenizer object at 0x7f255d632460>


In [12]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2 2701 2584    3    0    0    0    0    0    0    0    0    0    0]
[2701 2584    3    0    0    0    0    0    0    0    0    0    0    0]


In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 12000개로 선언했기 때문에, tokenizer.num_words의 값은 12000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [17]:
#총 데이터의 20% 를 평가 데이터셋
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

## Step 5. 인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!

잘 설계한 모델을 학습하려면, model.fit() 함수를 사용해야 합니다. model.fit() 함수에는 다양한 인자를 넣어주어야 하는데, 가장 기본적인 인자로는 데이터셋과 epochs가 있습니다. '5. 실습 (2) 인공지능 학습시키기'에서의 예시와 같이 말이죠.

model.fit(dataset, epochs=30)
하지만 model.fit() 함수의 epochs를 아무리 크게 넣는다 해도 val_loss 값은 2.2 아래로 떨어지지 않습니다. 이럴 경우는 batch size를 변경하는 것과 같이 model.fit() 함수에 다양한 인자를 넣어주면 해결될 수도 있습니다. 자세한 내용은 https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit 를 참고하세요!

Loss는 아래 제시된 Loss 함수를 그대로 사용하세요!

In [19]:
# #Loss
# # tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
# loss = tf.keras.losses.SparseCategoricalCrossentropy( 
#     from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.

In [21]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [22]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-5.00537899e-05,  1.29662847e-04, -2.20344327e-06, ...,
         -2.10190614e-04, -1.47213941e-04, -6.30898940e-05],
        [-3.44439468e-04,  3.65543528e-04,  1.44343692e-04, ...,
         -6.26830515e-05, -1.27065621e-04, -1.74422355e-04],
        [-3.62228398e-04,  4.04485152e-04, -1.94158914e-04, ...,
         -9.85962251e-05, -1.51114960e-04, -2.12538391e-04],
        ...,
        [ 1.12406444e-03,  1.83225691e-03, -3.48483620e-04, ...,
          1.75878557e-03, -1.69896742e-03, -1.81443128e-03],
        [ 1.22027274e-03,  1.92539929e-03, -2.60421075e-04, ...,
          1.92831771e-03, -1.78127200e-03, -2.08544917e-03],
        [ 1.29781291e-03,  1.97446579e-03, -1.48881401e-04, ...,
          2.07607518e-03, -1.80949003e-03, -2.31196079e-03]],

       [[-5.00537899e-05,  1.29662847e-04, -2.20344327e-06, ...,
         -2.10190614e-04, -1.47213941e-04, -6.30898940e-05],
        [-8.01212373e-05,  2.24258067e-04, -1

In [23]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [53]:
#모델 학습하기
#enc_train, enc_val, dec_train, dec_val
optimizer = tf.keras.optimizers.Adam()
#Loss
# tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.
model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train,dec_train,
                    epochs=10,
                    batch_size=256,
                    validation_data=(enc_val, dec_val))


Epoch 1/10
550/550 [==============================] - 108s 193ms/step - loss: 1.8727 - val_loss: 1.8834
Epoch 2/10
550/550 [==============================] - 109s 199ms/step - loss: 1.5967 - val_loss: 1.8734
Epoch 3/10
550/550 [==============================] - 110s 200ms/step - loss: 1.4417 - val_loss: 1.8810
Epoch 4/10
550/550 [==============================] - 110s 199ms/step - loss: 1.3244 - val_loss: 1.9099
Epoch 5/10
550/550 [==============================] - 110s 200ms/step - loss: 1.2335 - val_loss: 1.9426
Epoch 6/10
550/550 [==============================] - 110s 199ms/step - loss: 1.1611 - val_loss: 1.9768
Epoch 7/10
550/550 [==============================] - 110s 200ms/step - loss: 1.1034 - val_loss: 2.0177
Epoch 8/10
550/550 [==============================] - 110s 200ms/step - loss: 1.0577 - val_loss: 2.0471
Epoch 9/10
550/550 [==============================] - 110s 200ms/step - loss: 1.0213 - val_loss: 2.0810
Epoch 10/10
550/550 [==============================] - 110s 200m

과대적합되는 모습인것같다. 에폭을 줄여야 할것같긴하다.

In [55]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    while True:
        # 입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        # 예측된 값 중 가장 높은 확률인 word index를 뽑기
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 예측된 word index를 문장 뒤에 붙임
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [59]:
results = model.evaluate(enc_val,  dec_val, verbose=2) #val을 평가하니 이렇게 나왔다.

1099/1099 - 13s - loss: 2.1106


In [58]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you <end> '

In [60]:
generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=15)

'<start> i hate to see her go <end> '

In [62]:
generate_text(model, tokenizer, init_sentence="<start> you kill me", max_len=15)

'<start> you kill me , real nigga hate me <end> '

In [64]:
generate_text(model, tokenizer, init_sentence="<start> what", max_len=15)

'<start> what you want nixga what you what you want nixga <end> '

In [65]:
generate_text(model, tokenizer, init_sentence="<start> who", max_len=15)

'<start> who s that chick ? who s that chick ? <end> '

In [66]:
generate_text(model, tokenizer, init_sentence="<start> which", max_len=15)

'<start> which is funny because i just saw nick the other day <end> '

In [68]:
#모델 학습하기
#enc_train, enc_val, dec_train, dec_val
optimizer = tf.keras.optimizers.Adam()
#Loss
# tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.
model.compile(loss=loss, optimizer=optimizer)
Lyric =model.fit(dataset,
                    epochs=10,
                    )

Epoch 1/10
686/686 [==============================] - 120s 172ms/step - loss: 1.2244
Epoch 2/10
686/686 [==============================] - 124s 181ms/step - loss: 1.1081
Epoch 3/10
686/686 [==============================] - 125s 182ms/step - loss: 1.0509
Epoch 4/10
686/686 [==============================] - 124s 181ms/step - loss: 1.0114
Epoch 5/10
686/686 [==============================] - 124s 181ms/step - loss: 0.9825
Epoch 6/10
686/686 [==============================] - 125s 182ms/step - loss: 0.9622
Epoch 7/10
686/686 [==============================] - 125s 182ms/step - loss: 0.9479
Epoch 8/10
686/686 [==============================] - 125s 182ms/step - loss: 0.9348
Epoch 9/10
686/686 [==============================] - 125s 182ms/step - loss: 0.9251
Epoch 10/10
686/686 [==============================] - 124s 180ms/step - loss: 0.9162


In [69]:
results1 = model.evaluate(enc_val,  dec_val, verbose=2) #val을 평가하니 이렇게 나왔다.

1099/1099 - 12s - loss: 0.9121


데이터셋을 대상으로한 val_loss는 0.91이나왔다. (하지만 val에서 데이터가 겹치니 무의미한것같기도 하다.)

In [72]:
generate_text(model, tokenizer, init_sentence="<start> who", max_len=15)

'<start> who s that chick ? who s that chick ? <end> '

하지만, 제대로 출력이 된다는것에서 같은 성능인것 같기도하다.

실제로 있는대사를 따온것같다.

인종차별하는 머신러닝의 모습

# [결과 및 회고]
이번 프로젝트에서는 작사가 AI를 만들어 보았습니다.
결과적으로, validation loss =2.1106 까지 줄이는 모델을 만들 수 있었습니다.
또한, generate_text 함수를 사용해 실제로 문장이 문맥에 맞게 잘 형성되는지 확인해보았는데,
생각보다 작곡을 잘 해내서 놀랐습니다.
아래는 작곡의 예시입니다.

* <start> i love you <end> '
* '<start> i hate to see her go <end> '
* '<start> you kill me , real nigga hate me <end> '
    등등이 있습니다.
    
개인적으로 모델 dataset으로 하는건 틀린 방법같습니다. 
    
    model.fit(enc_train,dec_train,
                    epochs=10,
                    batch_size=256,
                    validation_data=(enc_val, dec_val))
    
    이방법이 맞는 방법 같습니다. 검증데이터와 학습데이터를 분리했기 떄문입니다.
    
역시 공부가 더필요한 부분입니다.
    
* inbedding_size의 좋은 시작점은 50-300 범위에 있다. 그러나 특정 작업에 대한 최적의 값을 찾기 위해 다른 inbedding_size 값으로 실험하는 것이 일반적이다.
   
* vocab_size는 단순히 어휘의 고유한 정수 인덱스의 총 개수입니다.
 